In [ ]:
#default_exp training

In [ ]:
#export
from tqdm.auto import tqdm
from speechsep.core import *
from speechsep.masks import *
from speechsep.pipe import *
from speechsep.utils import *
from speechsep.data import *
from speechsep.imports import *
from speechsep.models import *

In [ ]:
#export
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#export
#fn = Path("../../data/clips/")
fn = Path("../../datawav/")
pipe = AudioPipe(fn)

# Dataset/DataLoader

In [ ]:
#export
class Tensorify(Transform):
    def encodes(self, x, cuda=True):
        tnsr = complex2real(x.data) if hasattr(x, "data") else complex2real(x)
        return torch.FloatTensor(tnsr).cuda() if cuda else torch.FloatTensor(tnsr)

class AudioDataset(Dataset):
    @delegates(AudioPipe)
    def __init__(self, fn, **kwargs):
        self.fn = fn
        self.pipe = AudioPipe(fn, **kwargs)
        self.n_samples = len(get_audio_files(fn))
    
    def __getitem__(self, index):
        x,y = self.pipe(index)
        x,y = Tensorify()(x),Tensorify()(y)
        return x,y
    
    def __len__(self):
        return self.n_samples

# Loss Function

In [ ]:
#export
def loss_func(x,y):
    loss = nn.MSELoss()
    min_loss = min([loss(x[i], y[i]) for i in range(len(x))])
    return min_loss

# Training Loop

In [ ]:
#export
bs = 16
shuffle=True
workers=2

In [ ]:
#export
dataset = AudioDataset(fn)
n = len(dataset)
train_ds, valid_ds, test_ds = torch.utils.data.random_split(dataset, [50, 25, n-75])
train_dl = DataLoader(dataset=train_ds, batch_size=bs, shuffle=shuffle, num_workers=workers)
valid_dl = DataLoader(dataset=valid_ds, batch_size=bs, shuffle=shuffle, num_workers=workers)
test_dl = DataLoader(dataset=test_ds, batch_size=bs, shuffle=shuffle, num_workers=workers)

dataiter = iter(train_dl)
data = dataiter.next()

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataset.py", line 257, in __getitem__
    return self.dataset[self.indices[idx]]
  File "<ipython-input-6-f5412aa2eca5>", line 16, in __getitem__
    x,y = Tensorify()(x),Tensorify()(y)
  File "/home/jupyter/fastcore/fastcore/transform.py", line 61, in __call__
    def __call__(self, x, **kwargs): return self._call('encodes', x, **k

In [ ]:
#export
model = U_Net(img_ch=2, output_ch=4).cuda()
model.train();

In [ ]:
#export
n_epochs = 1
n_samples = len(train_dl.dataset)
n_iter = math.ceil(n_samples/bs)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
#export
for epoch in range(n_epochs):
    for i, (xb, yb) in tqdm(enumerate(train_dl)):
        print(1)
        out = model(xb)
        print(2)
        mask1 = MaskcIRM(out[:,:2,:,:])
        mask2 = MaskcIRM(out[:,2:,:,:])
        sep = mask1*xb, mask2*xb
        loss = loss_func(sep, yb)
        print(3)
        #if (i+1)%5==0:
        #    print(f'epoch {epoch}: step {(i+1)/n_iter}')
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(4)
    print(f"epoch {epoch+1} Finished")